# 一、model定义

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()  # 初始化父类nn.Module
        
        # 第一个卷积层: 输入3通道(RGB)，输出16特征图，卷积核大小5x5
        # 输入为 32x32x3 的图像，经过卷积后变为 28x28x16
        self.conv1 = nn.Conv2d(3, 16, 5)
        
        # 第一个最大池化层: 窗口大小2x2，步长2
        # 将特征图从 28x28x16 缩小为 14x14x16
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # 第二个卷积层: 输入16特征图，输出32特征图，卷积核大小5x5
        # 输入为 14x14x16，经过卷积后变为 10x10x32
        self.conv2 = nn.Conv2d(16, 32, 5)
        
        # 第二个最大池化层: 窗口大小2x2，步长2
        # 将特征图从 10x10x32 缩小为 5x5x32
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # 第一个全连接层: 将展平的特征图(5x5x32=800个神经元)连接到120个神经元
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        
        # 第二个全连接层: 120个神经元连接到84个神经元
        self.fc2 = nn.Linear(120, 84)
        
        # 输出层: 84个神经元连接到10个神经元(CIFAR10的10个类别)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 输入图像经过第一个卷积层，再通过ReLU激活函数，再经过最大池化
        x = self.pool1(F.relu(self.conv1(x)))
        
        # 第一层池化的输出经过第二个卷积层，再通过ReLU激活函数，再经过最大池化
        x = self.pool2(F.relu(self.conv2(x)))
        
        # 将卷积层输出的特征图展平为一维张量，-1表示自动计算批次大小
        x = x.view(-1, 32 * 5 * 5)
        
        # 展平后的特征通过第一个全连接层和ReLU激活函数
        x = F.relu(self.fc1(x))
        
        # 第一个全连接层的输出通过第二个全连接层和ReLU激活函数
        x = F.relu(self.fc2(x))
        
        # 第二个全连接层的输出通过输出层(不使用激活函数，因为后续会使用交叉熵损失)
        x = self.fc3(x)
        
        # 返回模型输出(logits)，用于计算损失和预测类别
        return x

# 二、加载数据

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 定义预处理变换：将PIL图像转换为Tensor，并进行归一化处理
# 归一化使像素值范围从[0,1]变为[-1,1]，有助于模型训练
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 加载CIFAR10训练数据集，共50000张图像
# root指定数据集存储位置，transform应用上述预处理
# download=True表示如果本地没有则从网络下载
trainset = torchvision.datasets.CIFAR10(root='../data/cifar10', train=True, download=True, transform=transform)

# 创建训练数据加载器，以便批量读取数据
# batch_size=128: 每次读取128张图片
# shuffle=True: 随机打乱数据，防止模型学习到数据顺序
# num_workers=0: 不使用多进程加载数据
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)

# 加载CIFAR10测试数据集，共10000张图像
testset = torchvision.datasets.CIFAR10(root='../data/cifar10', train=False, download=True, transform=transform)

# 创建测试数据加载器
# batch_size=10000: 一次性加载全部测试集，方便评估
# shuffle=False: 不打乱测试数据顺序
testloader = torch.utils.data.DataLoader(testset, batch_size=10000, shuffle=False, num_workers=0)

# 获取测试数据集的迭代器，并获取一批测试数据
test_data_iter = iter(testloader)
test_images, test_labels = next(test_data_iter)

# CIFAR10数据集的10个类别名称
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 定义图像显示函数
def imshow(img):
    img = img / 2 + 0.5     # 反归一化，将像素值从[-1,1]恢复到[0,1]
    npimg = img.numpy()     # 将PyTorch Tensor转换为NumPy数组
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # 调整维度顺序以符合matplotlib要求
    plt.show()

# 获取测试集的迭代器
dataiter = iter(testloader)
images, labels = next(dataiter)

# 显示一批测试图像
imshow(torchvision.utils.make_grid(test_images))  # make_grid将多张图片拼成一个网格图像

# 打印前4个图像的标签
print(' '.join(f'{classes[test_labels[j]]:5s}' for j in range(4)))

# 三、开始训练

In [ ]:
net = LeNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(5):  # 训练5个轮次
    running_loss = 0.0  # 用于累计一定批次内的损失值
    for i, data in enumerate(trainloader, start=0):  # 遍历训练数据集，enumerate同时返回数据和数据索引
        # 获取输入数据和标签
        inputs, labels = data
        
        # 梯度清零，防止梯度累积
        optimizer.zero_grad()
        
        # 前向传播：将输入数据传入模型得到输出
        outputs = net(inputs)
        
        # 计算损失：比较模型输出与真实标签之间的差异
        loss = loss_fn(outputs, labels)
        
        # 反向传播：计算损失对各参数的梯度
        loss.backward()
        
        # 参数更新：根据梯度和学习率更新模型参数
        optimizer.step()
        
        # 累加当前批次的损失值
        running_loss += loss.item()
        
        # 每500批次评估一次模型性能并打印训练信息
        if i % 300 == 299:
            # 使用torch.no_grad()避免在评估时计算梯度，节省内存
            with torch.no_grad():
                # 对测试图像进行前向传播
                outputs = net(test_images)
                # 获取每个样本的预测类别（概率最高的类别索引）
                predict_y = torch.max(outputs, dim=1)[1]
                # 计算准确率：预测正确的样本数 / 总样本数
                accuracy = torch.eq(predict_y, test_labels).sum().item() / test_labels.size(0)
                # 打印当前轮次、批次、平均损失和测试准确率
                print(f'epoch: {epoch}, step: {i} loss: {running_loss / 300:.3f} test accuracy: {accuracy:.3f}')
                # 重置累计损失值
                running_loss = 0.0

# 训练结束提示
print('Finished Training')
# 保存训练好的模型参数到文件
torch.save(net.state_dict(), './lenet.pth')